## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# File to Load (Remember to change these)
city_data_to_load = os.path.join("Resources", "Weather_Database_D02.csv")

# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Phuket,TH,8.0000,98.2500,85.12,100,20,9.22,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,71.82,72,93,18.70,overcast clouds
2,2,Faya,SA,18.3851,42.4509,63.12,72,100,5.84,overcast clouds
3,3,Saint-Francois,FR,46.4154,3.9054,65.68,77,0,1.14,clear sky
4,4,Lagoa,PT,39.0500,-27.9833,73.98,90,15,4.90,light rain


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria, (Inputs: Min=75, Max=90)
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Phuket,TH,8.0000,98.2500,85.12,100,20,9.22,few clouds
8,8,Mana,GF,5.6592,-53.7767,77.47,89,69,8.48,broken clouds
9,9,Pangai,TO,-19.8000,-174.3500,80.71,74,20,17.27,few clouds
11,11,Kavieng,PG,-2.5744,150.7967,84.22,72,100,16.37,light rain
12,12,Tahta,EG,26.7693,31.5021,83.70,31,4,7.18,clear sky
16,16,Champerico,GT,14.3000,-91.9167,81.50,76,100,2.93,overcast clouds
19,19,Port Hueneme,US,34.1478,-119.1951,87.24,76,29,8.01,scattered clouds
23,23,Avarua,CK,-21.2078,-159.7750,77.05,61,40,10.36,scattered clouds
25,25,Bathsheba,BB,13.2167,-59.5167,83.01,89,40,19.57,scattered clouds
28,28,Kawalu,ID,-7.3817,108.2082,77.38,81,94,3.56,overcast clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database.csv    257
City                      257
Country                   257
Lat                       257
Lng                       257
Max Temp                  257
Humidity                  257
Cloudiness                257
Wind Speed                257
Current Description       257
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

WeatherPy_Database.csv    257
City                      257
Country                   257
Lat                       257
Lng                       257
Max Temp                  257
Humidity                  257
Cloudiness                257
Wind Speed                257
Current Description       257
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Phuket,TH,85.12,few clouds,8.0000,98.2500,
8,Mana,GF,77.47,broken clouds,5.6592,-53.7767,
9,Pangai,TO,80.71,few clouds,-19.8000,-174.3500,
11,Kavieng,PG,84.22,light rain,-2.5744,150.7967,
12,Tahta,EG,83.70,clear sky,26.7693,31.5021,
16,Champerico,GT,81.50,overcast clouds,14.3000,-91.9167,
19,Port Hueneme,US,87.24,scattered clouds,34.1478,-119.1951,
23,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
25,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,
28,Kawalu,ID,77.38,overcast clouds,-7.3817,108.2082,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Phuket,TH,85.12,few clouds,8.0000,98.2500,The Surin
8,Mana,GF,77.47,broken clouds,5.6592,-53.7767,LE SAMANA
9,Pangai,TO,80.71,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort
11,Kavieng,PG,84.22,light rain,-2.5744,150.7967,Nusa Island Retreat
12,Tahta,EG,83.70,clear sky,26.7693,31.5021,فندق الزيات
...,...,...,...,...,...,...,...
687,Jiazi,CN,78.93,overcast clouds,22.8779,116.0670,Hotel Jin Hao
688,Greer,US,75.33,light rain,34.9387,-82.2271,Quality Inn
689,Jhelum,PK,86.52,clear sky,32.9331,73.7264,Tulip Riverside Hotel And Restaurant Jehlum
690,Sibolga,ID,75.99,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))